## 1- Libraries

In [15]:
import os
import numpy as np
import tensorflow
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import models
from keras import layers
from keras import optimizers

## 2- Directories

In [2]:
base_dir = '/content/deep_learning_python/data/dogs-vs-cats/small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

## 3- Instantating VGG16 convent

In [3]:
conv_base = VGG16(
  weights = 'imagenet',
  include_top = False,
  input_shape = (150, 150, 3)
)

In [4]:
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 150, 150, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 150, 150, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 150, 150, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 75, 75, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 75, 75, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 75, 75, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 37, 37, 128)       0         
__________

## 4- Ways of proceed

At this point there are two ways of proceed:

* Fast feature extraction without data augmentation
* Feature extraction with data augmentation

### 4.1 Fast feature extraction

#### 4.1.1- Extraction 

In [5]:
datagen = ImageDataGenerator(rescale = 1./255)
batch_size = 20

In [10]:
def extract_features(directory, sample_count):
    features = np.zeros(
        shape = (sample_count, 4, 4, 512)
      )
    labels = np.zeros(shape = (sample_count))

    generator = datagen.flow_from_directory(
        directory,
        target_size = (150, 150),
        batch_size = batch_size,
        class_mode = 'binary'
      )
  
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i*batch_size:(i+1)*batch_size] = features_batch
        labels[i*batch_size:(i+1)*batch_size] = labels_batch
        i += 1
        if i*batch_size >= sample_count:
            break
    return features, labels

In [11]:
train_features, train_labels = extract_features(train_dir, 2000)

Found 2000 images belonging to 2 classes.


In [23]:
validation_features, validation_labels = extract_features(validation_dir, 1000)

Found 1000 images belonging to 2 classes.


In [13]:
test_features, test_labels = extract_features(test_dir, 1000)

Found 1000 images belonging to 2 classes.


#### 4.1.2- Reshape

In [21]:
train_features = np.reshape(train_features, (2000, 4* 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4* 4 * 512))
test_features =  np.reshape(test_features, (1000, 4* 4 * 512))

#### 4.1.3- Definig and training the densely connected classifier

In [18]:
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 4*4*512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))

In [19]:
model.compile(
  optimizer = optimizers.RMSprop(lr = 2e-5),
  loss = 'binary_crossentropy',
  metrics = ['acc']
)

In [22]:
history = model.fit(
  train_features,
  train_labels,
  epochs = 30,
  batch_size = 20,
  validation_data = (validation_features, validation_labels)
)

ValueError: Error when checking input: expected dense_1_input to have shape (8192,) but got array with shape (1,)